In [13]:
from keras.utils import to_categorical
from keras.preprocessing.image import load_img  # Corrected import
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np

In [14]:
TRAIN_DIR = 'images/train'
TEST_DIR = 'images/test'

In [17]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label,"completed")
    return image_paths,labels

In [18]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [19]:
print(train)

                                image     label
0            images/train\angry\0.jpg     angry
1            images/train\angry\1.jpg     angry
2           images/train\angry\10.jpg     angry
3        images/train\angry\10002.jpg     angry
4        images/train\angry\10016.jpg     angry
...                               ...       ...
28816  images/train\surprise\9969.jpg  surprise
28817  images/train\surprise\9985.jpg  surprise
28818  images/train\surprise\9990.jpg  surprise
28819  images/train\surprise\9992.jpg  surprise
28820  images/train\surprise\9996.jpg  surprise

[28821 rows x 2 columns]


In [20]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [27]:
print(test)
print(test['image'])

                              image     label
0       images/test\angry\10052.jpg     angry
1       images/test\angry\10065.jpg     angry
2       images/test\angry\10079.jpg     angry
3       images/test\angry\10095.jpg     angry
4       images/test\angry\10121.jpg     angry
...                             ...       ...
7061  images/test\surprise\9806.jpg  surprise
7062  images/test\surprise\9830.jpg  surprise
7063  images/test\surprise\9853.jpg  surprise
7064  images/test\surprise\9878.jpg  surprise
7065   images/test\surprise\993.jpg  surprise

[7066 rows x 2 columns]
0         images/test\angry\10052.jpg
1         images/test\angry\10065.jpg
2         images/test\angry\10079.jpg
3         images/test\angry\10095.jpg
4         images/test\angry\10121.jpg
                    ...              
7061    images/test\surprise\9806.jpg
7062    images/test\surprise\9830.jpg
7063    images/test\surprise\9853.jpg
7064    images/test\surprise\9878.jpg
7065     images/test\surprise\993.jpg
Name:

In [28]:
from tqdm.notebook import tqdm

In [29]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image,grayscale = True)
        img = np.array(img)
        features.append(img)
    features = np.array(features) 
    features = features.reshape(len(features),48,48,1)
    return features

In [30]:
train_features = extract_features(train['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

C:\Users\arghy\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\utils\image_utils.py:409: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn(


In [31]:
test_features = extract_features(test['image'])

  0%|          | 0/7066 [00:00<?, ?it/s]

In [32]:
x_train = train_features/255.0
x_test = test_features/255.0

In [33]:
from sklearn.preprocessing import LabelEncoder

In [35]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [36]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [37]:
y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)

In [44]:
model = Sequential()
# convolutional layers
model.add(Conv2D(128,kernel_size=(3,3), activation='relu',input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256,kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512,kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512,kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(7, activation='softmax'))

In [45]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = 'accuracy')

In [49]:
model.fit(x=x_train, y=y_train, batch_size=128, epochs=100, validation_data=(x_test, y_test))

Epoch 1/100


226/226 [==============================] - 109s 476ms/step - loss: 1.8235 - accuracy: 0.2415 - val_loss: 1.7935 - val_accuracy: 0.2638
Epoch 2/100
226/226 [==============================] - 94s 418ms/step - loss: 1.7835 - accuracy: 0.2571 - val_loss: 1.7012 - val_accuracy: 0.3002
Epoch 3/100
226/226 [==============================] - 93s 412ms/step - loss: 1.6657 - accuracy: 0.3322 - val_loss: 1.5066 - val_accuracy: 0.4130
Epoch 4/100
226/226 [==============================] - 93s 412ms/step - loss: 1.5200 - accuracy: 0.4055 - val_loss: 1.3711 - val_accuracy: 0.4750
Epoch 5/100
226/226 [==============================] - 92s 409ms/step - loss: 1.4449 - accuracy: 0.4423 - val_loss: 1.3060 - val_accuracy: 0.4962
Epoch 6/100
226/226 [==============================] - 93s 410ms/step - loss: 1.3922 - accuracy: 0.4653 - val_loss: 1.2707 - val_accuracy: 0.5195
Epoch 7/100
226/226 [==============================] - 92s 409ms/step - loss: 1.3504 - accuracy: 0.4830 - val_loss: 1.254

In [51]:
model_json = model.to_json()
with open("emotiondetector.json",'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")    

C:\Users\arghy\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [52]:
from keras.models import model_from_json

In [53]:
json_file = open("emotiondetector.json","r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("emotiondetector.h5")

In [54]:
label = ['angry','disgust','fear','happy','neytral','sad','surprise']

In [57]:
def ef(image):
    img = load_img(image, grayscale=True)
    feature = np.array(img)
    feature = feature.reshape(1, 48, 48, 1)
    return feature / 255.0

In [60]:
image = 'images/train/happy/24.jpg'
print("Original image is of angry")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)

Original image is of angry
1/1 [==============================] - 0s 32ms/step
model prediction is  happy
